In [ ]:
#| default_exp examples

# examples

> Logging example functions.

In [ ]:
#|hide
from nbdev.showdoc import *


In [ ]:
#|export
import logging
SIMPLE_FORMAT = logging.Formatter('"%(message)s"')
BETTER_FORMAT = logging.Formatter('"%(asctime)s",%(name)s,%(funcName)s,%(levelname)s,"%(message)s"')
logging.basicConfig(
    format='"%(message)s"',
    level=logging.INFO)


In [ ]:
#|export
def example_01() -> None:
    '''
    Example 01
    Print debugging to the screen using print()
    '''
    print('This debug message uses print.')



In [ ]:
example_01()

This debug message uses print.


The next example uses logging instead of print(). 
Note that this is almost as simple as print().  In the following examples, we'll see why logging is better than print!

In [ ]:
#|export

def example_02() -> None:
    '''
    Example 02
    Log to the screen with the root logger.

    '''
    
    rootlogger = logging.getLogger()
    old_level = rootlogger.getEffectiveLevel()
    rootlogger.setLevel(level=logging.DEBUG)
    logging.debug('This debug message uses the root logger.')
    logging.getLogger().setLevel(old_level)





In [ ]:
example_02()

"This debug message uses the root logger."


We can get the root logger from anywhere in our application with logging.getLogger()
We already configured the root logger in example_02, but we can modify its behavior here.
This illustrates two handy things about logging:

- logging provides access to the loggers from anywhere, so the developer does not have to search the code.
- We can change the level of messages to suppress debug messages.

In [ ]:
#|export
def example_03() -> None:
    '''
    Example 03
    Add formatting to the root logger.
    '''
    
    rootlogger = logging.getLogger()
    rootlogger.handlers[0].setFormatter(BETTER_FORMAT)
    
    logging.info('This uses the root logger with a custom format, adding context information to each message.')
    


In [ ]:
example_03()

"2022-08-17 10:11:46,132",root,example_03,INFO,"This uses the root logger with a custom format, adding context information to each message."


In [ ]:
#|export

def example_04() -> None:
    '''
    Example 04
    Use a named logger that has unique properties from the root logger.
    '''

    # Let's create a custom logger, my_logger.
    my_logger = logging.getLogger(__name__)
    my_logger.setLevel(logging.DEBUG)
    logging.info('This INFO message will display.')
    logging.debug('We will not see this because DEBUG is lower than the root level INFO.')

    my_logger = logging.getLogger(__name__)
    my_logger.debug('The __main__ logger is set to a level of DEBUG, so this message displays.')
    my_logger.info('Note the __main__ logger uses the format from root.')
    my_logger.handlers = []



In [ ]:
example_04()

"2022-08-17 10:11:46,168",root,example_04,INFO,"This INFO message will display."
"2022-08-17 10:11:46,168",__main__,example_04,DEBUG,"The __main__ logger is set to a level of DEBUG, so this message displays."
"2022-08-17 10:11:46,168",__main__,example_04,INFO,"Note the __main__ logger uses the format from root."


In [ ]:
#|export

def example_05() -> None:
    '''
    Example 05
    Log to a file with print()
    '''
    import sys

    with open(file='example_print.log', mode='a') as file:
        print('This is an example of writing to a logfile with print.', file=file)

In [ ]:
example_05()


In [ ]:
#|export

def example_06_get_logger(level:int=logging.INFO) -> None:
    '''
    Example 06
    Customize our logger with a file handler and a formatter.
    '''
    logger =  logging.getLogger(__name__)
    logger.handlers.clear()
    logger.propagate = False
    return logger
    
def example_06_configure_handler(
    filename:str=None, 
    format:logging.Formatter=SIMPLE_FORMAT, 
    level:int=logging.INFO) -> logging.Handler:
    

    if filename is not None:
        handler = logging.FileHandler(filename=filename, mode='a')
        handler.setFormatter(fmt=BETTER_FORMAT)
        handler.setLevel(level)
    else:
        handler = None
    return handler

def example_06():
    '''
    Example 06
    Create and customize a file logger with a logging.Handler that
    sends log messages to a file and add a custom formatter
    '''
    FILENAME='logs/example_06.log'

    logger = example_06_get_logger(level=logging.INFO) # the screen handler won't show debug messages

    logger.addHandler(
        example_06_configure_handler(format=BETTER_FORMAT, 
                                     filename=FILENAME, 
                                     level=logging.DEBUG)) # the file handler WILL show debug messages.

    logger.debug(f"Debug messages go to the file {FILENAME}")


In [ ]:
example_06()

In [ ]:
!cat logs/example_06.log

"2022-08-17 10:00:11,586",__main__,example_06,DEBUG,"Debug messages go to the file logs/example_06.log"
"2022-08-17 10:00:11,586",__main__,example_06,INFO,"Info messages go to the file logs/example_06.log and the screen."
"2022-08-17 10:01:21,863",__main__,example_06,DEBUG,"Debug messages go to the file logs/example_06.log"
"2022-08-17 10:10:45,833",__main__,example_06,DEBUG,"Debug messages go to the file logs/example_06.log"
"2022-08-17 10:11:46,245",__main__,example_06,DEBUG,"Debug messages go to the file logs/example_06.log"


In [ ]:
#|export
'''
Example 07
A logger that sends debug messages to the screen and info messages to a file
'''

FILENAME='logs/example_07.log'

def example_07_get_logger() -> logging.Logger:
    logger =  logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    logger.propagate = False
    return logger

def example_07_configure_file_handler(filename:str=FILENAME) -> logging.Handler:

    file_handler = logging.FileHandler(filename=filename, mode='a')
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(BETTER_FORMAT)
    return file_handler

def example_07_configure_screen_handler() -> logging.Handler:
    import sys

    screen_handler = logging.StreamHandler(stream=sys.stdout)
    screen_handler.setLevel(logging.DEBUG)
    screen_handler.setFormatter(SIMPLE_FORMAT)

    return screen_handler

def example_07():
    logger = example_07_get_logger()
    logger.handlers.clear()
    logger.addHandler(example_07_configure_screen_handler())
    logger.addHandler(example_07_configure_file_handler())

    logger.debug('debug messages just go to the screen.  The screen format is simply the message.')
    logger.info("Info messages go to screen and file. The file format has more information.")

In [ ]:
example_07()

"debug messages just go to the screen.  The screen format is simply the message."
"Info messages go to screen and file. The file format has more information."


In [ ]:
!cat logs/example_07.log

"2022-08-17 10:00:11,755",__main__,example_07,INFO,"Info messages go to screen and file. The file format has more information."
"2022-08-17 10:11:46,450",__main__,example_07,INFO,"Info messages go to screen and file. The file format has more information."
